# @ 주제 : 한국인 메이저리거의 현지 평가 분석

# 1. 가설
* 트위터 글에 나타난 한국인 메이저리거의 현지 평가가 긍정적인지 부정적인지 판단할 수 있다.

# 2. 연구방법
## 2.1 데이터 수집
* 트위터에서 영문으로 작성된 메이저리거 관련 글을 수집함
* 하지만, 한국인 메이저리거 관련 트윗의 수가 적고 그 중 정서가 드러난 글은 더욱 부족함
* 한국인 메이저리거 뿐만 아닌 메이저리거 전체 선수 관련 트윗을 검색하여 정서가 드러난다고 판단되는 300 document를 추출함
* 해당 document가 긍정적인 정서인 경우 1, 부정적인 정서인 경우 0으로 class를 분류함
* (+추가) 후반부에 SVM 분류기 사용을 위해 class를 긍정적인 정서를 1, 부정적인 정서를 -1로 수정하게 됨

> Code - twitter API를 이용한 트위터 해쉬태그 검색

In [2]:
import twitter

# twitter API 사용을 위한 4가지 토큰
consumer_key="R8a6YSb4yf77X27emnGcI9hKdYYAEDbSqK08ocg6s5Vz9yLnr6"
consumer_secret="NIM6wPYiI2S4rCRkAPZUpaCpa"
token_key="VKTOHImjOvE4Kp8ECj3T11zzorHuwxUImNnfWGuuVIaK9"
token_secret="2373074766-PL14cizo2IrNPcvcUI25kdi7KjZQMAdRnNx2SPN"

api = twitter.Twitter(auth=twitter.OAuth(token_secret, token_key,consumer_secret, consumer_key))
    
response=api.search.tweets(q="#park")
str = response['statuses']
print(str)

[{u'contributors': None, u'truncated': False, u'text': u'RT @siro_cam: \u98a8\u306b\u306a\u3073\u3044\u3066\n\n#portrait #girlsportrait #park #cameraman #SiROphoto #photographer #2016album #pho\u2026 https://t.co/Wc8xsvyIdF https://t\u2026', u'is_quote_status': False, u'in_reply_to_status_id': None, u'id': 744702918419374080L, u'favorite_count': 0, u'entities': {u'symbols': [], u'user_mentions': [{u'id': 1453124808, u'indices': [3, 12], u'id_str': u'1453124808', u'screen_name': u'siro_cam', u'name': u'\u767d\u77f3 \u51cc\u592a\uff08\u30b7\u30ed\uff09'}], u'hashtags': [{u'indices': [22, 31], u'text': u'portrait'}, {u'indices': [32, 46], u'text': u'girlsportrait'}, {u'indices': [47, 52], u'text': u'park'}, {u'indices': [53, 63], u'text': u'cameraman'}, {u'indices': [64, 74], u'text': u'SiROphoto'}, {u'indices': [75, 88], u'text': u'photographer'}, {u'indices': [89, 99], u'text': u'2016album'}, {u'indices': [100, 104], u'text': u'pho'}], u'urls': [{u'url': u'https://t.co/Wc8xsvyIdF', u'in

## 2.2 데이터 처리
* 수집한 document들을 단어 벡터로 변환하는 작업이 필요함
* 단어 벡터 중에서 관사나 이름 명사 등 정서와 관련없는 단어(stop word)를 필터링함
* 이 때, 해쉬태그를 포함한 단어도 정서를 나타내는 경우가 존재하여 해쉬태그 문자가 제거된 형태로 해당 word를 사용


> text parsing(영문자 대소문자로만 이루어진 word 중 길이가 1 이상인 word만 남김)

In [15]:
import numpy as np
import re

docArr = []
classArr = []
regEx = re.compile('[^a-zA-Z]*') #complie: 정규식을 두 번 이상 사용할 경우 효율을 위함. 영문자 대/소문자를 제외한 문자열을 저장

file = open('data/trainingDoc.txt')
for line in file.readlines():
    #for i in range(len(line)):
    listOfTokens = regEx.split(line) #영문자 대/소문자를 이외의 문자를 기준으로 word 분리
    docArr.append([tok.lower() for tok in listOfTokens if len(tok) > 1]) #길이가 1이상인 word를 소문자 형태로 list에 저장

file = open('data/trainingLabel.txt')
for line in file.readlines():
    curLine = line.strip()
    classArr.append(curLine)

print docArr
print classArr

[['is', 'manny', 'machado', 'the', 'best', 'fantasy', 'baseball', 'player', 'by', 'trey', 'rose', 'aka', 'dynasty', 'digest', 'http', 'www', 'dobberbaseball', 'com', 'is', 'manny', 'machado', 'the', 'best', 'fantasy', 'baseball', 'player', 'trout', 'harper', 'machado'], ['machado', 'is', 'off', 'to', 'hot', 'start', 'do', 'you', 'think', 'he', 'll', 'be', 'an', 'allstar', 'mlb', 'orioles', 'http', 'fb', 'me', 'nt', 'zf', 'pc'], ['manny', 'machado', 'is', 'freak', 'of', 'nature', 'dude', 'guy', 'just', 'makes', 'plays', 'that', 'no', 'one', 'else', 'would', 'dream', 'of', 'orioles', 'platinumglove', 'mannybeingmanny'], ['hey', 'birdland', 'let', 'just', 'use', 'this', 'as', 'an', 'excuse', 'to', 'bask', 'in', 'the', 'glory', 'that', 'machado', 'is', 'our', 'third', 'baseman'], ['machado', 'needs', 'to', 'be', 'signed', 'for', 'years', 'machado', 'orioles'], ['fantasy', 'recommendation', 'by', 'brett', 'talley', 'manny', 'machado', 'http', 'bit', 'ly', 'sycmdh', 'alarm'], ['machado', 'is

> Code - stop word 생성 및 필터링

In [7]:
stopWord = ['manny', 'machado', 'the', 'www', 'http', 'orioles', 'mlb', 'park', 'maeda', 'kenta', 'dodgers',
            'ladodgers', 'com', 'seunghwanoh', 'seung', 'hwan', 'cardinals', 'stlouis', 'twins', 'debut',
            'hyunsookim', 'kim', 'hyun', 'soo', 'baltimore', 'korean', 'tyler', 'colins', 'tigers', 'tylercollins'
            'keuchel', 'houstonastros', 'giants', 'jepsen', 'braves', 'cubs', 'redSox', 'greinke', 'puig',
            'molina', 'detroitTigers', 'a', 'an', 'for', 'by', 'and', 'of', 'or', 'to', 'is', 'are', 'has',
            'have', 'about', 'i', 'you', 'hyunjinryu', 'hyunjin', 'ryu', 'la', 'byeonghopark', 'byeongho', 
            'park', 'twins', 'minnesota', 'minnesotatwins', 'daeholee', 'daeho', 'lee', 'mariners', 'seattle',
            'seattlemariners','junghokang', 'jungho', 'kang', 'pirates', 'pittsburgh', 'pittsburghpirtes',
            'sinsoochoo', 'sinsoo', 'choo', 'rangers', 'texas', 'texasrangers', 'darvish', 'osaka', 'yuDarvish',
            'rangers', 'he', 'they', 'http', ' ', 'at', 'in', 'him', 'his', 'with', 'that', 'indians', 'royals',
            'SeungHwanOh', 'Cardinals']

#입력 받은 document에서 stop word를 제거하는 함수
def filterDoc(docArr):    
    wordArr = []
    for i in range(len(docArr)):
        temp = []
        for word in docArr[i]:
            word = word.lower() #word를 소문자로 변환
            if word not in stopWord: #stop word가 아닌경우만 저장
                temp.append(word)

        wordArr.append(temp)
    
    return wordArr

wordArr = filterDoc(docArr)
print wordArr

[['best', 'fantasy', 'baseball', 'player', 'trey', 'rose', 'aka', 'dynasty_digest', 'dobberbaseball', 'best', 'fantasy', 'baseball', 'player', 'trout', 'harper'], ['off', 'hot', 'start', 'do', 'think', 'll', 'be', 'allstar', 'fb', 'me', '5nt5zf7pc'], ['freak', 'nature', 'dude', 'guy', 'just', 'makes', 'plays', 'no', 'one', 'else', 'would', 'dream', 'platinumglove', 'mannybeingmanny'], ['hey', 'birdland', 'let', 'just', 'use', 'this', 'as', 'excuse', 'bask', 'glory', 'our', 'third', 'baseman'], ['needs', 'be', 'signed', '20', 'years'], ['fantasy', 'recommendation', 'brett', 'talley', 'bit', 'ly', '1sycmdh', 'alarm'], ['just', 'so', 'damn', 'good'], ['moment', 'when', 'your', 'happy', 'your', 'recording', 'game', 'on', 'fire'], ['back', 'back', 'homeruns', 'way', 'go', 'reimold', 'omazing', 'go', 'abc2news', 'ibackthebirds'], ['back', 'back', 'homers', 'let', 'go', 'os', 'homeruns', 'schoop'], ['homers', 'inning', 'reimold', 'scoop', 'mazing'], ['hr', 'giving', 'me', 'boost', 'fantasy', 

## 2.3 분류기 생성
* Naive Bayes 분류기와 SVM 분류기의 성능을 각각 비교하여 더 좋은 성능의 분류기를 선택함
* (+ 추가) 본 실험에서 사용된 SVM 분류기는 'Machine Learning in Action' 책의 smo 코드를 기반으로 작성됨
* training 및 test data는 사전확률을 맞추기 위해 300개의 document 중 232개의 document만을 사용함
* 232개의 document 중 210(90%)개를 traing set으로, 나머지 22(10%)개의 한국인 메이저리거 관련 document를 test set으로 사용
* training set은 긍정 105개, 부정 105개. test set은 긍정 15개, 부정 7개 사용

> Naive Bayes 분류기 training

In [35]:
import bayes
reload(bayes)

#training document에 존재하는 유일한 단어들의 list 및 그 갯수
myVocabList = bayes.createVocabList(wordArr)
print len(myVocabList)

trainArr = []
NBclassArr = []

#문서 내에 어휘목록의 단어가 존재하는지 여부를 숫자 벡터로 변환(myVocabList내에 존재하면 1, 존재하지 않으면 0)
for postinDoc in wordArr:
    trainArr.append(bayes.setOfWords2Vec(myVocabList, postinDoc))

#문자열(string) 형태의 class 벡터를 정수(int) 형태로 변환함
for strVec in classArr:
    intVec = int(strVec)
    NBclassArr.append(intVec)

p0V,p1V,pAb=bayes.trainNB0(trainArr,NBclassArr) #Naive Bayes 분류기 training

#training 오류 계산
errorCnt = 0
i = 0
for temp in trainArr:
    predict = bayes.classifyNB(temp,p0V,p1V,pAb) #Naive bayes 분류
    if np.sign(predict) != np.sign(int(NBclassArr[i])):
        errorCnt += 1
    print 'class is ', NBclassArr[i], 'classified as ', np.sign(predict), '\n'
    i += 1

print 'Total error rate : ', float(errorCnt)/ np.shape(trainArr)[0]

861
class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

class is  1 classified as  1 

clas

> SVM 분류기 training

In [20]:
import svmMLiA
reload(svmMLiA)

SVMclassArr = []

#SVM 적용을 위해 class를 1 or 0 형태가 아닌 1 or -1 형태로 변환함
for strVec in classArr:
    intVec = int(strVec)
    if intVec == 1:
        SVMclassArr.append(int(1))
    else:
        SVMclassArr.append(int(-1))
        
#b, alphas = svmMLiA.smoP(trainArr, SVMclassArr, 200, 0.001, 1000, ('rbf', 200)) #kernel version
b, alphas = svmMLiA.smoP(trainArr, SVMclassArr, 200, 0.001, 1000) #linear version
print b, alphas

L==H
fullSet, iter: 0 i:0, pairs changed 0
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 2
fullSet, iter: 0 i:3, pairs changed 3
fullSet, iter: 0 i:4, pairs changed 4
fullSet, iter: 0 i:5, pairs changed 5
fullSet, iter: 0 i:6, pairs changed 6
fullSet, iter: 0 i:7, pairs changed 7
fullSet, iter: 0 i:8, pairs changed 8
fullSet, iter: 0 i:9, pairs changed 9
fullSet, iter: 0 i:10, pairs changed 10
fullSet, iter: 0 i:11, pairs changed 11
fullSet, iter: 0 i:12, pairs changed 12
fullSet, iter: 0 i:13, pairs changed 13
fullSet, iter: 0 i:14, pairs changed 14
fullSet, iter: 0 i:15, pairs changed 14
fullSet, iter: 0 i:16, pairs changed 15
fullSet, iter: 0 i:17, pairs changed 15
fullSet, iter: 0 i:18, pairs changed 15
fullSet, iter: 0 i:19, pairs changed 15
fullSet, iter: 0 i:20, pairs changed 15
fullSet, iter: 0 i:21, pairs changed 15
fullSet, iter: 0 i:22, pairs changed 15
fullSet, iter: 0 i:23, pairs changed 15
fullSet, iter: 0 i:24, pairs changed 16
fullSet, iter: 

In [24]:
#training 결과를 통해 분류기 d(x) = wx + b 추출
ws = svmMLiA.calcWs(alphas, trainArr, SVMclassArr) #분류기의 w항 계산
#print ws
#print trainMat
trainMat = np.mat(trainArr)
SVMclassMat = np.mat(SVMclassArr).transpose()

svInd = np.nonzero(alphas.A > 0)[0] #서포트 벡터의 index를 리스트로 반환
sVs = trainMat[svInd]
labelSV = SVMclassMat[svInd]

print "there are %d Supprot Vectors" % np.shape(sVs)[0]

there are 161 Supprot Vectors


In [25]:
#training 오류 계산
errorCnt = 0
m = np.shape(trainMat)[0]
for i in range(m):
    #kernelEval = svmMLiA.kernelTrans(sVs, trainMat[i,:], ('rbf', 200))
    #predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b #kernel version
    predict = trainMat[i] * np.mat(ws) + b #linear version
    if np.sign(predict) != np.sign(int(SVMclassArr[i])):
        errorCnt += 1
    print 'class is ', SVMclassArr[i], 'classified as ', np.sign(predict), '\n'

print 'Total error rate : ', float(errorCnt)/ m

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

class is  1 classified as  [[ 1.]] 

c

## 2.4 각 분류기 테스트

> 1) NavieBayes 분류기로 테스트

In [28]:
#test를 위한 입력데이터 전처리
NBtestArr = []
testClassArr = []
file =open('data/testDoc.txt')

for line in file.readlines():
    lineArr = []
    curLine = line.strip().split('\t')
    
    listOfTokens = regEx.split(curLine[0])
    NBtestArr.append([tok.lower() for tok in listOfTokens if len(tok) > 1]) #길이가 1이상인 word를 소문자 형태로 list에 저장
    testClassArr.append(curLine[-1])

filteredArr = filterDoc(NBtestArr)


#test 오류 계산
errorCnt = 0
i = 0
for doc in filteredArr:
    temp = np.array(bayes.setOfWords2Vec(myVocabList, doc))
    classified = bayes.classifyNB(temp,p0V,p1V,pAb) #Naive bayes 분류기 수행
    
    #에러율 계산
    if classified != int(testClassArr[i]):
        errorCnt += 1
    print 'class is ', testClassArr[i], 'classified as ', classified, '\n'
    i += 1
    
print 'Total error rate : ', float(errorCnt)/len(filteredArr)    

the word: power is not in my Vocabulary!
the word: pitching is not in my Vocabulary!
the word: mph is not in my Vocabulary!
the word: strike is not in my Vocabulary!
class is  1 classified as  0 

the word: reliever is not in my Vocabulary!
the word: koreatimesus is not in my Vocabulary!
the word: relief is not in my Vocabulary!
class is  1 classified as  0 

the word: nickname is not in my Vocabulary!
the word: anything is not in my Vocabulary!
class is  1 classified as  1 

the word: stunning is not in my Vocabulary!
the word: galleryelenash is not in my Vocabulary!
the word: londonartfair is not in my Vocabulary!
the word: impermanence is not in my Vocabulary!
the word: series is not in my Vocabulary!
the word: allmatterwilldecay is not in my Vocabulary!
class is  1 classified as  0 

class is  1 classified as  1 

the word: day is not in my Vocabulary!
the word: day is not in my Vocabulary!
class is  1 classified as  1 

the word: byung is not in my Vocabulary!
the word: entire is 

> 2) SVM 분류기로 테스트

In [30]:
#test를 위한 입력데이터 전처리
SVMtestArr = []
for doc in filteredArr:
    temp = np.array(bayes.setOfWords2Vec(myVocabList, doc))
    SVMtestArr.append(temp)    
testMat = np.mat(SVMtestArr)

for i in range(len(testClassArr)):
    if testClassArr[i] == '0' :
        testClassArr[i] = '-1'


#test 오류 계산
errorCnt = 0
m = np.shape(testMat)[0]
for i in range(m):
    #kernelEval = svmMLiA.kernelTrans(sVs, testMat[i,:], ('rbf', 200))
    #predict = kernelEval.T * np.multiply(labelSV, alphas[svInd]) + b #kernel version
    predict = trainMat[i] * np.mat(ws) + b #linear version
    if np.sign(predict) != np.sign(int(testClassArr[i])):
        errorCnt += 1
    print 'class is ', testClassArr[i], 'classified as ', np.sign(predict), '\n'

print 'Total error rate : ', float(errorCnt)/ m

the word: power is not in my Vocabulary!
the word: pitching is not in my Vocabulary!
the word: mph is not in my Vocabulary!
the word: strike is not in my Vocabulary!
the word: reliever is not in my Vocabulary!
the word: koreatimesus is not in my Vocabulary!
the word: relief is not in my Vocabulary!
the word: nickname is not in my Vocabulary!
the word: anything is not in my Vocabulary!
the word: stunning is not in my Vocabulary!
the word: galleryelenash is not in my Vocabulary!
the word: londonartfair is not in my Vocabulary!
the word: impermanence is not in my Vocabulary!
the word: series is not in my Vocabulary!
the word: allmatterwilldecay is not in my Vocabulary!
the word: day is not in my Vocabulary!
the word: day is not in my Vocabulary!
the word: byung is not in my Vocabulary!
the word: entire is not in my Vocabulary!
the word: successful is not in my Vocabulary!
the word: roster is not in my Vocabulary!
the word: byung is not in my Vocabulary!
the word: moon is not in my Vocabul

- <b>linear일 때, 훈련 오류율 0%, 검사오류율 23%, 지지 벡터의 개수 161개</b>
- RBF, 0.1일 때, 훈련 오류율 0%, 검사오류율 77%, 지지 벡터의 개수 209개
- RBF, 1일 때, 훈련 오류율 0%, 검사오류율 27%, 지지 벡터의 개수 209개
- RBF, 5일 때, 훈련 오류율 1%, 검사오류율 36%, 지지 벡터의 개수 177개
- RBF, 10일 때, 훈련 오류율 15%, 검사오류율 63%, 지지 벡터의 개수 166개
- RBF, 50일 때, 훈련 오류율 37%, 검사오류율 77%, 지지 벡터의 개수 121개
- RBF, 100일 때, 훈련 오류율 17%, 검사오류율 68%, 지지 벡터의 개수 116개
- RBF, 200일 때, 훈련 오류율 33%, 검사오류율 77%, 지지 벡터의 개수 110개

### > SVM 분류기 중, 선형 SVM 분류기를 사용했을 때 가장 좋은 분류 결과를 보임

# 3. 결론
#### 22개의 test set을 이용한 분류 결과
- Naive Bayes 분류기: 훈련 오류율 4%, 검사오류율 55%
- 선형 SVM 분류기: 훈련 오류율 0%, 검사오류율 23%

#### 따라서, 가장 좋은 성능을 보이는 선형 SVM 분류기를 본 실험의 분류기로 사용

#### > 선형 SVM 분류기를 사용하여 트위터 상의 메이저리거 관련 글의 정서를 77% 확률로 판단할 수 있다

#### 보완할 점
- 복수/단수 처리나 be동사 처리 등의 Stemming 작업이 필요함
- 더 많은 수의 트위터 글을 사용하여 분류 결과를 비교해 볼 필요가 있음

